# metadata exploration

In [1]:
import pandas as pd
datas = []
for i in range(0,10):
    datas.append(pd.read_csv("../data/raw/metadata" + str(i) + ".csv", header=None, index_col=0))
all_met =pd.concat(datas, sort=False)
import ast
all_met[1] = all_met[1].apply(ast.literal_eval)
all_met[["abstract", "description", "keywords", "crashed"]] = all_met[1].apply(pd.Series)
#crashed is always false
all_met = all_met.drop(['crashed'], axis=1)

headers_dom = ['url', 'section', 'code']
dom = pd.read_csv('../data/raw/20181018 Domain+WZ2008 Hackathon INOBAS.csv', sep=';', names=headers_dom, encoding='latin-1', skiprows=1)

dom.head()

,url,section,code
0,www.hoberg-driesch.de,L,68.31
1,www.hgdf.de,K,64.20
2,www.gasag.de,D,35.30
3,www.ergodirekt.de,K,65.11
4,www.wahler.de,M,70.10


In [2]:
dom.shape

(10779, 3)

In [3]:
all_df = pd.concat([dom, all_met], axis=1, sort=False)

In [4]:
def map_keywords(keywords):
    return [y.strip() for y in keywords.split(",")]
all_df['keywords'] = all_df['keywords'].apply(map_keywords)

In [5]:
all_df.t

,url,section,code,1,abstract,description,keywords
0,www.hoberg-driesch.de,L,68.31,(Qualitätsstahlrohre von Hoberg & Driesch. Von...,Qualitätsstahlrohre von Hoberg & Driesch. Von ...,Hoberg & Driesch GmbH & Co. KG ist eines der f...,"[Röhrengrosshandel, Starkwandrohre, Normalwand..."
1,www.hgdf.de,K,64.20,"(, , , False)",,,[]
2,www.gasag.de,D,35.30,"(, Bei der GASAG finden Sie die richtigen Stro...",,Bei der GASAG finden Sie die richtigen Strom- ...,"[Strom, Erdgas, Wärme, Privatkunden, Stromvert..."
3,www.ergodirekt.de,K,65.11,"(, ERGO Direkt ist meistgewählter Direktversic...",,ERGO Direkt ist meistgewählter Direktversicher...,[]
4,www.wahler.de,M,70.10,"(, , , False)",,,[]


In [6]:
all_df['keywords']

0        [Röhrengrosshandel, Starkwandrohre, Normalwand...
1                                                       []
2        [Strom, Erdgas, Wärme, Privatkunden, Stromvert...
3                                                       []
4                                                       []
5        [MSA, MSA safety, MSAsafety, safety products, ...
6                               [NÜRNBERGER, Versicherung]
7        [Customer Care Services, Kundenservices, Kunde...
8                                                       []
9        [Goldhofer Unternehmensgruppe, Goldhofer Compa...
10       [Bürobedarf, Textmarker, Faber-Castell 9000 Bl...
11                                                      []
12       [Biotest, Forschung, Plasmaprodukte, Biotherap...
13       [Vattenfall, Strom, Gas, Tarifwechsel, Mein Va...
14                                                      []
15                                                      []
16                                                      

In [7]:
# -*- coding: utf-8 -*-

import http.client, urllib.parse, uuid, json

# **********************************************
# *** Update or verify the following values. ***
# **********************************************

def translate_real(text):
    
    if len(text) == 0:
        return
    
    # Replace the subscriptionKey string value with your valid subscription key.
    subscriptionKey = 'b1a0198a38694e8f91d7af82a2710ae3'

    host = 'api.cognitive.microsofttranslator.com'
    path = '/translate?api-version=3.0'

    # Translate to German and Italian.
    params = "&to=en";

    def __translate (content):

        headers = {
            'Ocp-Apim-Subscription-Key': subscriptionKey,
            'Content-type': 'application/json',
            'X-ClientTraceId': str(uuid.uuid4())
        }

        conn = http.client.HTTPSConnection(host)
        conn.request ("POST", path + params, content, headers)
        response = conn.getresponse ()
        return response.read ()

    requestBody = [{
        'Text' : text,
    }]
    content = json.dumps(requestBody, ensure_ascii=False).encode('utf-8')
    result = __translate (content)

    # Note: We convert result, which is JSON, to and from an object so we can pretty-print it.
    # We want to avoid escaping any Unicode characters that result contains. See:
    # https://stackoverflow.com/questions/18337407/saving-utf-8-texts-in-json-dumps-as-utf8-not-as-u-escape-sequence
    output = json.loads(result)

    return output[0]['translations'][0]['text']

NameError: name 'all_df' is not defined

In [10]:
translate_helper(['hallo', 'guten Tag'])

[['hallo', 'Hello'], ['guten Tag', 'Good day']]

In [30]:
from multiprocessing import Pool as ProcessPool
from tqdm import tqdm
import numpy as np

def translate_helper(words):
    words = list(filter(lambda word: len(word) > 0, words))
    if len(words) == 0:
        return []
    return list(map(lambda word: {word: translate_real(word)}, words))

keywords = list(all_df['keywords'])
print(keywords)

[['Röhrengrosshandel', 'Starkwandrohre', 'Normalwandrohre', 'Gewinderohre', 'Kesselrohre', 'Hohlprofile', 'Präzisions\xadstahl\xadrohre', 'HPZ-Rohre', 'Wälzlager\xadstahl\xadrohre', 'Feinkornbaustahlrohre', 'Maschinenbaustahlrohre'], [''], ['Strom', 'Erdgas', 'Wärme', 'Privatkunden', 'Stromvertrag', 'Erdgasvertrag', 'Gas', 'Gasvertrag', 'Berlin', 'Brandenburg', 'Gasversorgung', 'Gastarif', 'Stromtarif', 'Tarif', 'Geschäftskunden', 'Kunden', 'Verbrauch', 'Energie', 'Energievertrag', 'Energieversorgung', 'Versorgungsunternehmen', 'Strom wechseln', 'Gas wechseln'], [''], [''], ['MSA', 'MSA safety', 'MSAsafety', 'safety products', 'msa global'], ['NÜRNBERGER', 'Versicherung'], ['Customer Care Services', 'Kundenservices', 'Kundenkontakt Management', 'Call Center', 'Communication Center', 'Web Kommunikation', 'Web Services', 'Web Service first', 'Agentur Services', 'Web Agentur', 'Social Media Services', 'Outsourcing', 'D+S Gruppe', 'D+S 360°', 'new-how', 'newhow', 'Webever', 'D+S communicat

In [31]:
workers = ProcessPool(12)
count = 0
results = np.array([])
with tqdm(total=len(keywords)) as pbar:
    for result in tqdm(workers.imap_unordered(translate_helper, keywords)):
        count += len(result)
        results = np.append(results, result)
        pbar.update()





  0%|          | 0/10779 [00:00<?, ?it/s]




0it [00:00, ?it/s]



  0%|          | 15/10779 [00:00<03:38, 49.20it/s]




15it [00:00, 57.27it/s]



  0%|          | 22/10779 [00:00<03:36, 49.69it/s]




22it [00:00, 54.94it/s]



  0%|          | 26/10779 [00:00<04:52, 36.72it/s]




26it [00:00, 38.90it/s]



  0%|          | 30/10779 [00:00<04:47, 37.36it/s]




30it [00:00, 38.93it/s]



  0%|          | 34/10779 [00:00<04:54, 36.46it/s]




34it [00:00, 37.48it/s]



  0%|          | 38/10779 [00:01<08:58, 19.93it/s]




38it [00:01, 19.67it/s]



  0%|          | 41/10779 [00:01<09:22, 19.08it/s]




41it [00:01, 19.48it/s]



  0%|          | 45/10779 [00:01<08:08, 21.95it/s]




45it [00:01, 22.31it/s]



  0%|          | 51/10779 [00:01<07:38, 23.42it/s]




51it [00:01, 23.41it/s]



  1%|          | 55/10779 [00:02<08:58, 19.91it/s]




55it [00:01, 20.12it/s]



  1%|          | 58/10779 [00:02<10:32, 16.95it/s]




58it [00:02, 16.94it/s]



  1%|          | 60/10779 

In [68]:
results[0]

{'NÜRNBERGER': 'Nuremberg'}

In [127]:
find('Starkwandrohre')

''

In [161]:
from tqdm import tqdm

c = 0

def find(key):
    global c
    for r in results:
        if key in r:
            if r[key] == key:
                c += 1
                return ''
            return r[key]
        
def not_none(key):
    if key is None:
        return ''
    
    return key

def gen_en_list_order(german_list):
    res = []
    with tqdm(total=len(german_list)) as pbar:
        for l in tqdm(german_list):
            res_l = list(map(lambda ger: find(ger), l))
            res_l = list(map(lambda ger: not_none(ger), res_l))
            res_l = list(filter(lambda w: len(w) > 0, res_l))
            if (len(res_l) == 0):
                res_l = ['']
            res.append(res_l)
    return res

In [162]:
b = gen_en_list_order(list(all_df['keywords']))





  0%|          | 0/10779 [00:00<?, ?it/s]




  0%|          | 0/10779 [00:00<?, ?it/s]




  0%|          | 25/10779 [00:00<00:44, 243.26it/s]




  0%|          | 50/10779 [00:00<00:43, 243.95it/s]




  1%|          | 74/10779 [00:00<00:44, 238.66it/s]




  1%|          | 95/10779 [00:00<00:47, 225.47it/s]




  1%|          | 117/10779 [00:00<00:48, 220.86it/s]




  1%|▏         | 138/10779 [00:00<00:49, 214.89it/s]




  2%|▏         | 164/10779 [00:00<00:46, 226.51it/s]




  2%|▏         | 185/10779 [00:00<00:48, 219.22it/s]




  2%|▏         | 206/10779 [00:00<00:49, 211.61it/s]




  2%|▏         | 227/10779 [00:01<00:51, 206.38it/s]




  2%|▏         | 248/10779 [00:01<00:53, 198.20it/s]




  2%|▏         | 268/10779 [00:01<00:56, 186.18it/s]




  3%|▎         | 287/10779 [00:01<00:57, 183.13it/s]




  3%|▎         | 309/10779 [00:01<00:54, 191.34it/s]




  3%|▎         | 334/10779 [00:01<00:50, 205.36it/s]




  3%|▎         | 355/10779 [00:01<00:50, 204.74it/s]


In [164]:
c

28075

In [163]:
b

[['Precision Steel Tubes', 'HPZ Pipes', 'Roller bearing Tubes'],
 [''],
 ['Electricity',
  'Natural gas',
  'Heat',
  'Private',
  'Gas supply',
  'Tariff',
  'Electricity tariff',
  'Rate',
  'Business',
  'Customers',
  'Consumption',
  'Energy',
  'Energy supply',
  'Utilities',
  'Change current',
  'Change gas'],
 [''],
 [''],
 [''],
 ['Nuremberg', 'Insurance'],
 ['Customer service',
  'Customer contact Management',
  'Web Communication',
  'Agency services',
  'Web Agency',
  'D + S Group',
  'D + S 360 °',
  'Website development',
  'Mobile app Development'],
 [''],
 ['Goldhofer Group of Companies',
  'Transport Airport Technology Technology Service Trailers Sattelanghänger Heavy load Modules Trailers Semitrailers heavy-duty-modules Germany conventional rodless luggage hauler aircraft Tug Arts conventional TOWBARLESS cargo'],
 ['Office supplies',
  'Highlighter',
  'Faber-Castell 9000 pencils',
  'Crayons',
  'Pen',
  'Filler',
  'Fountain pen',
  'Markers',
  'Rollerballs',
  '

In [165]:
all_df['keywords_en'] = b

In [186]:
g = all_df[['url','keywords_en']]

a = dict(sorted(g.values.tolist()))
np.save(a, a)